In [28]:
import os
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from scipy import stats
from tqdm import tqdm

from assistments_workbench.config_reader import config
from assistments_workbench.models.model import classification_model

In [29]:
data = pd.read_csv(os.path.join(config.get('localfiles', 'data_path'), 'arrs_data.csv'))

In [30]:
data['grade_diff_2'] = np.nan
data['ms_bin'] = np.nan

In [31]:
data.describe()

,id,student_class_id,class_grade,grade_diff,adaptive_mode,mastered,delay_days,work_delay_in_day,ms,class_performance,...,skill_id,prereq_skill_id,prereq_easiness,prereq_performance,easiness,right,wrong,correct,grade_diff_2,ms_bin
count,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,...,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,21425.000000,0,0
mean,570159.633559,33109.643174,7.479347,-0.050035,0.535543,0.992999,6.077993,16.284294,4.752159,0.707702,...,207.173676,176.095776,0.711619,0.808622,0.708009,3.559253,1.190805,0.701190,NaN,NaN
std,100392.987895,2675.788911,1.371097,0.529575,0.498747,0.083382,1.781270,26.939130,3.302097,0.062715,...,60.857489,55.756447,0.093623,0.218598,0.146376,1.350446,2.195581,0.457747,NaN,NaN
min,389087.000000,22906.000000,5.000000,-5.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.477352,...,46.000000,10.000000,0.452082,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,491757.000000,31432.000000,7.000000,0.000000,0.000000,1.000000,6.000000,2.000000,3.000000,0.684806,...,196.000000,130.000000,0.661031,0.666667,0.617647,3.000000,0.000000,0.000000,NaN,NaN
50%,549286.000000,31883.000000,8.000000,0.000000,1.000000,1.000000,7.000000,7.000000,3.000000,0.725898,...,217.000000,212.000000,0.701277,0.833333,0.741860,3.000000,0.000000,1.000000,NaN,NaN
75%,657761.000000,36454.000000,8.000000,0.000000,1.000000,1.000000,7.000000,18.000000,6.000000,0.744366,...,259.000000,213.000000,0.773080,1.000000,0.811111,4.000000,2.000000,1.000000,NaN,NaN
max,820106.000000,40977.000000,12.000000,10.000000,1.000000,1.000000,7.000000,268.000000,79.000000,0.847858,...,296.000000,296.000000,0.946102,1.000000,1.000000,25.000000,54.000000,1.000000,NaN,NaN


In [32]:
data = data.drop_duplicates(['id', 'prereq_skill_id','prereq_easiness'], keep='first')

In [33]:
data.describe()

,id,student_class_id,class_grade,grade_diff,adaptive_mode,mastered,delay_days,work_delay_in_day,ms,class_performance,...,skill_id,prereq_skill_id,prereq_easiness,prereq_performance,easiness,right,wrong,correct,grade_diff_2,ms_bin
count,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,...,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,20564.000000,0,0
mean,566536.336462,33002.983077,7.483807,-0.046878,0.532241,0.992706,6.110679,16.487794,4.689068,0.709437,...,207.686588,176.569977,0.711412,0.810482,0.709818,3.540897,1.145983,0.715814,NaN,NaN
std,100241.793871,2653.785596,1.371187,0.506381,0.498972,0.085097,1.750721,26.989248,3.135947,0.061337,...,61.173924,55.723398,0.094021,0.217528,0.144406,1.314566,2.070086,0.451037,NaN,NaN
min,389087.000000,22906.000000,5.000000,-5.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.477352,...,46.000000,10.000000,0.452082,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,491142.250000,31432.000000,7.000000,0.000000,0.000000,1.000000,6.000000,2.000000,3.000000,0.686752,...,196.000000,130.000000,0.661031,0.681818,0.620961,3.000000,0.000000,0.000000,NaN,NaN
50%,548494.000000,31862.000000,8.000000,0.000000,1.000000,1.000000,7.000000,7.000000,3.000000,0.725898,...,217.000000,212.000000,0.701277,0.833333,0.742044,3.000000,0.000000,1.000000,NaN,NaN
75%,657217.500000,36454.000000,8.000000,0.000000,1.000000,1.000000,7.000000,18.000000,5.000000,0.744366,...,259.000000,213.000000,0.779803,1.000000,0.811467,4.000000,1.000000,1.000000,NaN,NaN
max,820106.000000,40977.000000,12.000000,10.000000,1.000000,1.000000,7.000000,268.000000,79.000000,0.847858,...,296.000000,296.000000,0.946102,1.000000,1.000000,25.000000,54.000000,1.000000,NaN,NaN


In [34]:
# data = data.drop(data[(data['mastered'] ==1) & (data['ms'] < 3)].index)
# data.describe()

In [35]:
data_0 = data[data['mastered'] == 0]
data_1 = data[data['mastered'] == 1]
data_1 = data[data['ms'] >= 3]

In [36]:
group_by_col = 'ms'
grouped = data_1.groupby(group_by_col)

In [37]:
for name, group in grouped:
    if name >=3 and name <=4:
        data_1.loc[(data_1[group_by_col] == name),'ms_bin'] = 1
    elif name >=5 and name <=6:
        data_1.loc[(data_1[group_by_col] == name),'ms_bin'] = 2
    else:
        data_1.loc[(data_1[group_by_col] == name),'ms_bin'] = 3

In [38]:
data = pd.concat([data_0, data_1])
data.loc[(data['mastered'] == 0),'ms_bin'] = 0
data.describe()

,id,student_class_id,class_grade,grade_diff,adaptive_mode,mastered,delay_days,work_delay_in_day,ms,class_performance,...,skill_id,prereq_skill_id,prereq_easiness,prereq_performance,easiness,right,wrong,correct,grade_diff_2,ms_bin
count,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,...,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000,0,20361.000000
mean,565565.921910,32998.107411,7.477236,-0.047787,0.530720,0.987427,6.142773,16.587790,4.754285,0.709003,...,207.064093,176.253475,0.711408,0.809368,0.712406,3.572811,1.177251,0.717941,NaN,1.486567
std,100134.672163,2652.693206,1.375117,0.507048,0.499068,0.111425,1.706971,27.053813,3.136324,0.061474,...,61.195012,55.849685,0.094005,0.218317,0.142913,1.298108,2.094446,0.450013,NaN,0.772774
min,389087.000000,22906.000000,5.000000,-5.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.477352,...,46.000000,10.000000,0.452082,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000
25%,491085.000000,31432.000000,7.000000,0.000000,0.000000,1.000000,6.000000,2.000000,3.000000,0.684806,...,196.000000,130.000000,0.661031,0.666667,0.625407,3.000000,0.000000,0.000000,NaN,1.000000
50%,548359.000000,31862.000000,8.000000,0.000000,1.000000,1.000000,7.000000,7.000000,3.000000,0.725898,...,217.000000,212.000000,0.701277,0.833333,0.742857,3.000000,0.000000,1.000000,NaN,1.000000
75%,657189.000000,36454.000000,8.000000,0.000000,1.000000,1.000000,7.000000,19.000000,6.000000,0.744366,...,259.000000,213.000000,0.779803,1.000000,0.812536,4.000000,2.000000,1.000000,NaN,2.000000
max,820106.000000,40977.000000,12.000000,10.000000,1.000000,1.000000,7.000000,268.000000,79.000000,0.847858,...,296.000000,296.000000,0.946102,1.000000,1.000000,25.000000,54.000000,1.000000,NaN,3.000000


In [39]:
group_by_col = 'grade_diff'
grouped = data.groupby(group_by_col)

In [40]:
for name, group in grouped:
    if name == 0:
        # on grade
        data.loc[(data[group_by_col] == name),'grade_diff_2'] = 0
    else:
        if name > 0:
            # above grade
             data.loc[(data[group_by_col] == name),'grade_diff_2'] = 1
        elif name == -1:
            # one year ago
            data.loc[(data[group_by_col] == name),'grade_diff_2'] = 2
        else:
            # longer ago
            data.loc[(data[group_by_col] == name),'grade_diff_2'] = 3

In [41]:
data.describe()
data.to_csv('cleaned.csv')

In [42]:
pfa_data = data[['skill_id', 'easiness', 'right', 'wrong', 'correct']]

In [43]:
pfa_data.describe()

,skill_id,easiness,right,wrong,correct
count,20361.000000,20361.000000,20361.000000,20361.000000,20361.000000
mean,207.064093,0.712406,3.572811,1.177251,0.717941
std,61.195012,0.142913,1.298108,2.094446,0.450013
min,46.000000,0.000000,0.000000,0.000000,0.000000
25%,196.000000,0.625407,3.000000,0.000000,0.000000
50%,217.000000,0.742857,3.000000,0.000000,1.000000
75%,259.000000,0.812536,4.000000,2.000000,1.000000
max,296.000000,1.000000,25.000000,54.000000,1.000000


In [44]:
label = 'correct'
all_features = pfa_data.columns.tolist()
all_features.remove(label)
predictors = all_features
categorical_features = ['skill_id']
k = 5

In [45]:
model = LogisticRegression()
print classification_model(model, pfa_data, predictors, label, categorical_features, k)

(0.72930880053664793, 0.1339348909170221, 0.41887549671861846, 0.74166371625496763)


In [46]:
label = 'correct'
all_features = data.columns.tolist()
all_features.remove(label)
all_features.remove('id')
all_features.remove('grade_diff')
all_features.remove('ms')
all_features.remove('right')
all_features.remove('wrong')
all_features.remove('mastered')
predictors = all_features
categorical_features = ['student_class_id', 'adaptive_mode', 'skill_id', 'prereq_skill_id', 'grade_diff_2', 'ms_bin']

In [47]:
# model = ensemble.GradientBoostingClassifier()
model = LogisticRegression()

In [48]:
print classification_model(model, data, predictors, label, categorical_features, k)

(0.76419662586592618, 0.18549770125084228, 0.40622565587274523, 0.75958980675366117)


In [49]:
single_feature_performance = pd.DataFrame(index = range(len(all_features)), columns = ['feature', 'AUC', 'R2', 'r'])

In [50]:
for n, feature in enumerate(all_features):
    if feature in categorical_features:
        cf = [feature]
        r = np.nan
    else:
        cf = None
        r, _ = stats.pearsonr(data[feature], data[label])
    
    a,b,c,d = classification_model(model, data, [feature], label, cf, k)
    
    single_feature_performance.ix[n,'feature'] = feature
    single_feature_performance.ix[n,'AUC'] = a
    single_feature_performance.ix[n,'R2'] = b
    single_feature_performance.ix[n,'r'] = r
    

In [51]:
single_feature_performance = single_feature_performance.sort_values(by='AUC', axis=0, ascending=False).reset_index(drop=True)

single_feature_performance

,feature,AUC,R2,r
0,class_assignment_performance,0.725472,0.131216,0.364248
1,easiness,0.712877,0.116485,0.344288
2,student_class_id,0.633531,0.0454206,NaN
3,ms_bin,0.596729,0.0340525,NaN
4,prereq_skill_id,0.585119,0.0219684,NaN
5,skill_id,0.58397,0.0232362,NaN
6,class_performance,0.579911,0.016049,0.126669
7,delay_days,0.570571,0.0146218,0.122441
8,prereq_performance,0.55492,0.00804164,0.0896779
9,class_grade,0.549536,0.010339,-0.103835


In [ ]:
num_features = 5
num_combinations = 500

In [ ]:
feature_column_names = ['feature'+str(x+1) for x in range(num_features)]
feature_combinations_performance = pd.DataFrame(index=range(num_combinations), columns = feature_column_names + ['AUC', 'R2'])

for combination in tqdm(range(num_combinations)):
    random_features = sorted(np.random.choice(len(all_features), num_features, replace=False))
    
    # store the feature names
    random_feature_names = [all_features[x] for x in random_features]
    for i in range(len(random_features)):
        feature_combinations_performance.ix[combination, feature_column_names[i]] = random_feature_names[i]
    
    cf = []
    for f in random_feature_names:
        if f in categorical_features:
            cf.append(f)
    if len(cf) == 0:
        cf = None
        
    a,b,c,d = classification_model(model, data, random_feature_names, label, cf, k)
    feature_combinations_performance.ix[combination,'AUC'] = a
    feature_combinations_performance.ix[combination,'R2'] = b


 40%|█████████████████▌                          | 2/5 [00:00<00:00, 16.53it/s]

In [ ]:
feature_combinations_performance = feature_combinations_performance.sort_values(by='AUC', axis=0, ascending=False).reset_index(drop=True)
feature_combinations_performance.ix[:10,:]